In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import cv2


paths = pathlib.Path('./Optical Micrographs').glob('01-H.jpg')
#paths = pathlib.Path('./Optical Micrographs').glob('*H.jpg')
paths_sorted = [x for x in paths]
paths_sorted.sort()

def toBW(img):
    (thresh, blackAndWhiteImage) = cv2.threshold(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), 127, 255, cv2.THRESH_BINARY)
    return blackAndWhiteImage

originals = [cv2.imread(str(path)) for path in paths_sorted]
bws = [toBW(original) for original in originals]
#bws = [toBW(cv2.imread(str(path))) for path in paths_sorted]

#Question for Zhouer: any suggestion on how to measure roundness?

In [19]:
def getCountours(img):
    contours, hierarchy = cv2.findContours(~img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def convertToDataArray(contour, index):
    area = cv2.contourArea(contour)
    perimiter = cv2.arcLength(contour, True)
    (cx, cy), (w, h), c = cv2.minAreaRect(contour)
    return [index+1, area, perimiter, w, h, cx, cy]

def paintContours(img, cnts):
    for cnt in cnts:
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        cv2.drawContours(img,[box],0,(0,0,255),2)
def flatten(t):
    return [item for sublist in t for item in sublist]
contours = [getCountours(i) for i in bws]

#paintContours(originals[0], contours[0])
#cv2.imwrite("bwconverted.pdf", bws[0])
#cv2.imwrite("contours.pdf", originals[0])

#We want source image, area, length, height, location
# image [28]
#   pores [800]
#       attributes [7]
data = flatten([[convertToDataArray(cnt, index) for cnt in item] for index, item in enumerate(contours)])
data = filter(lambda x: x[1] < 1000000 and x[1] > 0, data)

In [17]:
arr = np.array([np.array(d) for d in data])
np.savetxt('pores-filtered.csv', arr, delimiter=',', header="file, area, circumference, width, height, center X, center Y")

In [23]:
cv2.drawContours(originals[0],contours[0],-1,(0,0,255),2)
cv2.imwrite("contours.pdf", originals[0])

True

In [33]:
# Structure of return data is
#[[Next, Previous, First_Child, Parent]]
# Next step: print a tree
# Idea: identify largest contour (whole image?) using conservative contrast.
# Identify largest contour inside (build object).
# Eliminate everything outside of the contour
# Identify pores using less conservative contrast
# Draw pores on original image to check validity
def getHierarchy(img):
    contours, hierarchy = cv2.findContours(~img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return hierarchy

print(getHierarchy(bws[0]))
print(cv2.contourArea(contours[0][265]))

[[[  1  -1  -1  -1]
  [  2   0  -1  -1]
  [  3   1  -1  -1]
  ...
  [873 871  -1 265]
  [ -1 872  -1 265]
  [ -1 265  -1 109]]]
109540869.0
